In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import ConvLSTM2D, Attention, Dense, Flatten
from tensorflow.keras.utils import Sequence
import os
from scipy.io import loadmat
from tensorflow.keras import Model
from PIL import Image
import pickle

In [ ]:
labels_path = './Labels_MERL_Shopping_Dataset/'
results_path = './Results_MERL_Shopping_Dataset/DetectedActions/'
videos_path = './Videos_MERL_Shopping_Dataset/'

In [ ]:
x_train_path = videos_path+'train/'
y_train_path = 'train_y.pkl'

In [ ]:
x_test_path = videos_path + 'test/'
y_test_path = 'test_y.pkl'

In [ ]:
x_val_path = videos_path + '/val/'
y_val_path = 'val_y.pkl'

In [ ]:
class DataGenerator(Sequence):
    
    def __init__(self, x_path, y_path = None, to_fit = True,  seq_len = 30):
        self.x_path = x_path        
#         self.batch_size = batch_size
        self.to_fit = to_fit
        self.list_X = os.listdir(self.x_path)
        self.seq_len = seq_len
        if to_fit:
            self.y_path = y_path
            self.dict_Y = self.get_y(y_path)
    
    
    def __len__(self):
        return len(self.list_X)
    
    
    def __getitem__(self, index):
        images_folder = self.list_X[index]
        images_list = sorted(os.listdir(self.x_path + images_folder))
        all_frames = []
        for img in images_list:
            all_frames.append(np.array(Image.open(x_train_path+images_folder+'/'+img)))
        
        X = self.stack_frames(all_frames)
        
        if self.to_fit:
            key = images_folder.split('_')[:2]
            key = '_'.join(key)
            Y = np.array(self.dict_Y(key))
            return X, Y[30:]
        
        return X
    
    def get_y(self, path):
        with open(path, 'rb') as pickle_file:
            y_dict = pickle.load(pickle_file)
        return y_dict 
    
    def stack_frames(self, frames):
        stacked_frames = []
        for i in range(len(frames) - self.seq_len):
            end = i + 30
            stacked_frames.append(frames[i:end])
        
        return np.stack(stacked_frames)

In [ ]:
training_generator = DataGenerator(x_train_path ,y_path = y_train_path)
validation_generator = DataGenerator(x_val_path ,y_path = y_val_path)
testing_generator = DataGenerator(x_test_path ,y_path = y_test_path)

In [ ]:
class MyCL_Model(Model):
    
    def __init__(self):
        super(MyCL_Model, self).__init__()
        self.convlstm = ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_first'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                       , padding='same', return_sequences=True)

        
    def call(self, inputs):
        x = self.convlstm(inputs)
        x = self.attention(x)
        return self.classifier(x)

In [ ]:
model = MyCL_Model()
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
model.fit_generator(generator = training_generator, validation_data=validation_generator)